In [1]:
import pymongo
import subprocess
import os
from config import DB_NAME, FEAT_3_COLLECTION_NAME, FEAT_30_COLLECTION_NAME

ModuleNotFoundError: No module named 'config'

In [2]:
# # If you need to reset mongoDB collections, run this cell
client = pymongo.MongoClient('localhost')
db = client[DB_NAME]
collection_list = db.list_collection_names()

for i in collection_list:
    mycol = db[i]
    print(mycol)
    mycol.drop()
    collection_list = db.list_collection_names()
    print(collection_list)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Music_db'), 'feat_3')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Music_db'), 'feat_30')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Music_db'), 'fs.files')
Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Music_db'), 'fs.chunks')


In [3]:
cwd_bytes = subprocess.check_output("pwd")
cwd = cwd_bytes.decode("utf-8").rstrip("\n") + "/"

In [4]:
features_3_path  = "Data/features_3_sec.csv"
features_30_path = "Data/features_30_sec.csv"
genres_path = "Data/genres_original/"
images_path = "Data/images_original/"

In [5]:
# Roadmap:
# 1: List genre directories in genres path and images path
# 2.using ls, add to list to create list of files in all genres genres and images
# 3. Then, Add each file to mongodb
byte_genres = subprocess.check_output(["ls", cwd + genres_path])
genre_folder = byte_genres.decode("utf-8").split("\n")
genre_folder.pop(-1)
genre_folder

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [6]:
byte_images = subprocess.check_output(["ls", cwd + images_path])
images_folder = byte_images.decode("utf-8").split("\n")
images_folder.pop(-1)
images_folder

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [7]:
# For both genres and images folders: list all files
# in each folder. Add to python list, iterate through this list,
# put in mongodatabase using mongofiles
for folder in genre_folder:
    byte_files = subprocess.check_output(["ls", cwd + genres_path + folder])
    files = byte_files.decode("utf-8").split("\n")
    files.pop(-1)

    for file in files:
        file_path = genres_path + folder + "/" + file
        os.system(f"mongofiles -d={DB_NAME} put " + file_path)

In [8]:
for folder in images_folder:
    byte_files = subprocess.check_output(["ls", cwd + images_path + folder])
    files = byte_files.decode("utf-8").split("\n")
    files.pop(-1)

    for file in files:
        file_path = images_path + folder + "/" + file
        os.system(f"mongofiles -d={DB_NAME} put " + file_path)

In [9]:
# Saves csv files as Mongodb collections
os.system(f"mongoimport -d {DB_NAME} -c {FEAT_3_COLLECTION_NAME} --type csv --file {features_3_path} --headerline")
os.system(f"mongoimport -d {DB_NAME} -c {FEAT_30_COLLECTION_NAME} --type csv --file {features_30_path} --headerline")

0